# Task 4: Signing a Message with PQC Digital Signatures

## Learning Objectives

Upon completing this task, you should be able to:

1.  **Understand the purpose of digital signatures:** Providing authenticity, integrity, and non-repudiation.
2.  **Identify** ML-DSA (CRYSTALS-Dilithium, FIPS 204) and SLH-DSA (SPHINCS+, FIPS 205) as NIST-standardized Post-Quantum Cryptography (PQC) signature algorithms.
3.  **Understand the process of PQC signature key generation:** Creating a distinct public/private key pair specifically for signing (different from KEM keys).
4.  **Perform message signing:** Use a PQC private key to generate a digital signature for a given message.
5.  **Recognize** that the output of the signing process is a "signature," which is a block of data.
6.  **Prepare** the necessary components (public key, original message, signature) for the verification process (Task 5).

## Understanding the Task and the Code

Digital signatures are a cornerstone of secure communication and data validation. They allow a recipient to verify:
*   **Authenticity:** That the message indeed came from the claimed sender (who possesses the corresponding private signing key).
*   **Integrity:** That the message has not been altered in transit since it was signed.
*   **Non-repudiation:** The sender cannot easily deny having signed the message, as only they should have access to the private signing key.

In this task, you will:
1.  **Generate a PQC Signature Key Pair:** Similar to KEMs, PQC signature schemes use a public/private key pair. The private key is kept secret by the signer, while the public key is distributed to anyone who needs to verify signatures. These keys are *different* from the keys used for encryption/encapsulation in Tasks 1-3.
2.  **Sign a Message:** The signing process takes the message and the signer's **private key** as input and produces a **digital signature**. This signature is unique to both the message content and the private key used.

**Key Generation (`on_generate_sig_keys_button_clicked` function):**
                                                                                                                                                             
*   An `oqs.Signature(selected_sig_alg)` object is created.
*   `public_key_sig = sig.generate_keypair()`: Generates the public/private key pair for the chosen signature algorithm.
*   `secret_key_sig = sig.export_secret_key()`: Retrieves the private key.
*   The keys are stored (in memory for this lab) and displayed in Base64.

**Message Signing (`on_sign_message_button_clicked` function):**
*   It checks if signature keys have been generated.
*   The message is taken from the input text area and converted to bytes (UTF-8 encoded).
*   An `oqs.Signature(signing_alg)` object is created (or re-used if state management was more complex).
*   `signature_bytes = sig_signer.sign(message_bytes, secret_key=_task4_data["sig_private_key_bytes"])`: This is the core signing operation. It uses the **private signing key** (stored in `_task4_data`) to produce the signature for the `message_bytes`.
*   The resulting `signature_bytes` are Base64 encoded and displayed.
 
**Your Interaction:**

1.  Run the Python code cell below.
2.  Select a "Signature Algorithm" from the dropdown (e.g., a variant of ML-DSA or SLH-DSA).
3.  Click "Generate Signature Keys." Observe the public and private keys.
4.  Enter or modify the text in the "Message to Sign" area.
5.  Click "Sign Message."
6.  Observe the "Digital Signature" generated in the output area.
7.  Read the status messages and details about the keys and signature.

## Next Steps

The three crucial pieces of information you'll need from this task for **Task 5 (Verifying a Signature)** are:
1.  The **Signer's Public Key** (Base64 encoded).
2.  The **Original Message** (the exact plaintext you entered).
3.  The **Digital Signature** (Base64 encoded).

Make sure you copy these accurately! Task 5 will demonstrate how anyone with the public key can verify that the message is authentic and unaltered.

---
**Now, run the code cell below to perform Task 4.**

In [4]:
import oqs
import ipywidgets as widgets
from IPython.display import display, HTML as IPHTML
import base64
import os

# --- Standardized Signature Algorithms ---
all_supported_sigs = oqs.get_supported_sig_mechanisms()
standardized_sig_options = []

# ML-DSA (Dilithium) variants - FIPS 204
# Common liboqs names: Dilithium2, Dilithium3, Dilithium5
# FIPS names: ML-DSA-44, ML-DSA-65, ML-DSA-87
dilithium_variants = {
    "ML-DSA-44": "Dilithium2", # NIST Security Level 2
    "ML-DSA-65": "Dilithium3", # NIST Security Level 3
    "ML-DSA-87": "Dilithium5", # NIST Security Level 5
}
for fips_name, common_name in dilithium_variants.items():
    if fips_name in all_supported_sigs:
        standardized_sig_options.append(fips_name)
    elif common_name in all_supported_sigs:
        standardized_sig_options.append(common_name)

# SLH-DSA (SPHINCS+) variants - FIPS 205
# Focusing on SHA2, 's' (small signature/slow sign) and 'f' (fast sign/larger signature) variants.
# liboqs names often append "-simple" or "-robust". We'll try to be flexible.
sphincs_variants_templates = [ # (FIPS_base, liboqs_base, levels)
    ("SLH-DSA-SHA2-", "SPHINCS+-SHA2-", ["128s", "128f", "192s", "192f", "256s", "256f"]),
    ("SLH-DSA-SHAKE-", "SPHINCS+-SHAKE-", ["128s", "128f", "192s", "192f", "256s", "256f"]), # SHAKE variants
]
sphincs_suffixes = ["-simple", "-robust", ""] # Order of preference for suffixes

for fips_base, liboqs_base, levels in sphincs_variants_templates:
    for level in levels:
        fips_level_name = fips_base + level
        liboqs_level_name_base = liboqs_base + level
        found_sphincs = False
        if fips_level_name in all_supported_sigs: # Direct FIPS name match
            standardized_sig_options.append(fips_level_name)
            found_sphincs = True
        else:
            for suffix in sphincs_suffixes:
                liboqs_full_name = liboqs_level_name_base + suffix
                if liboqs_full_name in all_supported_sigs:
                    standardized_sig_options.append(liboqs_full_name)
                    found_sphincs = True
                    break
        if found_sphincs:
            continue


# Remove duplicates and sort
supported_sig_algs_task4 = sorted(list(set(standardized_sig_options)))

# Absolute fallback if dynamic checks yield nothing (unlikely if liboqs is installed correctly)
if not supported_sig_algs_task4:
    # Check a few common ones directly
    fallbacks = ["ML-DSA-65", "Dilithium3", "SLH-DSA-SHA2-128s-simple", "SPHINCS+-SHA2-128s-simple"]
    for fb_alg in fallbacks:
        if fb_alg in all_supported_sigs:
            supported_sig_algs_task4.append(fb_alg)
    if not supported_sig_algs_task4 and all_supported_sigs:
        supported_sig_algs_task4 = [all_supported_sigs[0]] # Pick at least one if any exist
    elif not supported_sig_algs_task4:
        supported_sig_algs_task4 = ["Dilithium3"] # Last resort if OQS is non-responsive


# --- UI Elements ---
header_task4 = IPHTML("<h2>Task 4: Signing a Message with PQC Digital Signatures</h2>")
description_task4 = IPHTML(f"""
<p>This task demonstrates how to create a digital signature for a message using NIST-standardized Post-Quantum Cryptography (PQC) signature algorithms:
<b>ML-DSA (Dilithium)</b> from FIPS 204 or <b>SLH-DSA (SPHINCS+)</b> from FIPS 205.</p>
<p>A digital signature provides <b>authenticity</b> (proof of who signed it), <b>integrity</b> (proof the message hasn't changed), and <b>non-repudiation</b> (the signer cannot easily deny signing it).</p>
<ol>
    <li>Select a PQC signature algorithm from the dropdown. These are variants of ML-DSA or SLH-DSA.</li>
    <li>Click 'Generate Signature Keys' to create a new public/private key pair for signing.</li>
    <li>Enter the message you wish to sign in the 'Message to Sign' text area.</li>
    <li>Click 'Sign Message'. The PQC signature will be generated using the private key.</li>
</ol>
<p>The Signer's Public Key, Original Message, and the Digital Signature will be needed in Task 5 for verification.</p>
<p><i>Available standardized algorithms in your liboqs: {', '.join(supported_sig_algs_task4) if supported_sig_algs_task4 else 'None found, check liboqs.'}</i></p>
""")

sig_alg_dropdown_task4 = widgets.Dropdown(
    options=supported_sig_algs_task4,
    value=supported_sig_algs_task4[0] if supported_sig_algs_task4 else None,
    description='Signature Algorithm:',
    disabled=not supported_sig_algs_task4,
    style={'description_width': 'initial'}
)

generate_sig_keys_button_task4 = widgets.Button(
    description='Generate Signature Keys',
    button_style='info',
    tooltip='Generate a new public/private key pair for the selected signature algorithm',
    icon='key'
)

sig_public_key_output_task4 = widgets.Textarea(
    value='',
    placeholder='Signature Public Key (Base64) will appear here. Copy for Task 5.',
    description='Signer\'s Public Key:',
    layout={'width': '95%', 'height': '100px'},
    disabled=True,
    style={'description_width': 'initial'}
)

sig_private_key_output_task4 = widgets.Textarea(
    value='',
    placeholder='Signature Private Key (Base64) will appear here. Used for signing.',
    description='Signer\'s Private Key:',
    layout={'width': '95%', 'height': '100px'},
    disabled=True,
    style={'description_width': 'initial'}
)

message_to_sign_input_task4 = widgets.Textarea(
    value='This message will be signed using PQC!',
    placeholder='Enter the message you want to sign.',
    description='Message to Sign:',
    layout={'width': '95%', 'height': '80px'},
    disabled=False,
    style={'description_width': 'initial'}
)

sign_message_button_task4 = widgets.Button(
    description='Sign Message',
    button_style='primary',
    tooltip='Sign the message using the generated private key',
    icon='pencil-alt' # FontAwesome 5 icon for signing/editing
)

signature_output_task4 = widgets.Textarea(
    value='',
    placeholder='PQC Digital Signature (Base64) will appear here. Copy for Task 5.',
    description='Digital Signature:',
    layout={'width': '95%', 'height': '100px'},
    disabled=True,
    style={'description_width': 'initial'}
)

status_output_task4 = widgets.Output(layout={'width': '95%'})

next_steps_info_task4 = widgets.HTML(value="""
    <hr>
    <h4>Outputs for Task 5 (Verification):</h4>
    <p>To verify this signature in Task 5, you will need to copy the following (all Base64 encoded where applicable):</p>
    <ol>
        <li>The <b>Signer's Public Key</b>.</li>
        <li>The <b>Original Message</b> (as plaintext, ensure it's exactly what was signed).</li>
        <li>The <b>Digital Signature</b>.</li>
    </ol>
    <p>Ensure you copy each entire string accurately.</p>
    """,
    layout={'width': '95%'}
)

_task4_data = {
    "sig_public_key_bytes": None,
    "sig_private_key_bytes": None,
    "message_signed_str": None,
    "signature_bytes": None,
    "algorithm": None
}

def on_generate_sig_keys_button_clicked(b):
    selected_sig_alg = sig_alg_dropdown_task4.value
    sig_public_key_output_task4.value = ""
    sig_private_key_output_task4.value = ""
    signature_output_task4.value = "" # Clear previous signature
    _task4_data["sig_public_key_bytes"] = None
    _task4_data["sig_private_key_bytes"] = None
    _task4_data["signature_bytes"] = None
    _task4_data["algorithm"] = None

    with status_output_task4:
        status_output_task4.clear_output()
        if not selected_sig_alg:
            print("Error: No signature algorithm selected or available in liboqs.")
            return
        
        print(f"Generating signature key pair for {selected_sig_alg}...")
        try:
            with oqs.Signature(selected_sig_alg) as sig:
                public_key_sig = sig.generate_keypair()
                secret_key_sig = sig.export_secret_key()

                _task4_data["sig_public_key_bytes"] = public_key_sig
                _task4_data["sig_private_key_bytes"] = secret_key_sig
                _task4_data["algorithm"] = selected_sig_alg # Store the actual algorithm used

                sig_public_key_output_task4.value = base64.b64encode(public_key_sig).decode('utf-8')
                sig_private_key_output_task4.value = base64.b64encode(secret_key_sig).decode('utf-8')
                
                print(f"Successfully generated signature key pair for {selected_sig_alg}.")
                print(f"  Algorithm Details: {sig.details['name']}")
                print(f"  Claimed NIST level: {sig.details['claimed_nist_level']}")
                print(f"  Public Key Length: {sig.details['length_public_key']} bytes")
                print(f"  Private Key Length: {sig.details['length_secret_key']} bytes")
                print(f"  Max Signature Length: {sig.details['length_signature']} bytes")
                print("You can now enter a message and click 'Sign Message'.")

        except oqs.MechanismNotSupportedError:
            print(f"Error: Signature algorithm '{selected_sig_alg}' is not supported by your liboqs build.")
        except Exception as e:
            print(f"An unexpected error occurred during key generation: {e}")
            import traceback
            traceback.print_exc()

def on_sign_message_button_clicked(b):
    message_str = message_to_sign_input_task4.value
    signature_output_task4.value = ""
    _task4_data["signature_bytes"] = None # Clear previous

    with status_output_task4:
        status_output_task4.clear_output()
        
        current_dropdown_alg = sig_alg_dropdown_task4.value
        if not _task4_data["sig_private_key_bytes"] or not _task4_data["algorithm"]:
            print("Error: Please generate signature keys first using the 'Generate Signature Keys' button.")
            return
        
        # Ensure consistency between generated keys and selected algorithm for signing
        # This is important if the user changes the dropdown after generating keys
        if _task4_data["algorithm"] != current_dropdown_alg:
            print(f"Warning: The selected algorithm '{current_dropdown_alg}' differs from the algorithm of the generated keys ('{_task4_data['algorithm']}').")
            print(f"Signing will proceed using the algorithm of the generated keys: '{_task4_data['algorithm']}'.")
            # Optionally, update the dropdown to reflect the key's algorithm
            # sig_alg_dropdown_task4.value = _task4_data["algorithm"]
        
        signing_alg = _task4_data["algorithm"] # Use the algorithm for which keys were generated

        if not message_str.strip():
            print("Error: Please enter a message to sign.")
            return

        print(f"Signing message using {signing_alg}...")
        try:
            message_bytes = message_str.encode('utf-8')
            
            # Re-initialize Signature object. Provide the secret_key to the sign method.
            with oqs.Signature(signing_alg, _task4_data["sig_private_key_bytes"]) as sig_signer:
                # For oqs-python, the sign method itself needs the secret_key passed explicitly
                # if the KEM object wasn't "primed" by keygen or import_secret_key in the *same instance*.
                # The _task4_data stores the raw bytes.
                signature_bytes = sig_signer.sign(message_bytes)

            _task4_data["message_signed_str"] = message_str # Store original message
            _task4_data["signature_bytes"] = signature_bytes
            
            signature_output_task4.value = base64.b64encode(signature_bytes).decode('utf-8')
            print(f"Message signed successfully with {signing_alg}.")
            print(f"  Signature length: {len(signature_bytes)} bytes.")
            print("The Public Key, original message (from the input field), and this signature are needed for Task 5 (Verification).")

        except oqs.MechanismNotSupportedError: 
            print(f"Error: Signature algorithm '{signing_alg}' is not supported by your liboqs build (this is unexpected if keys were generated).")
        except Exception as e:
            print(f"An unexpected error occurred during signing: {e}")
            import traceback
            traceback.print_exc()

generate_sig_keys_button_task4.on_click(on_generate_sig_keys_button_clicked)
sign_message_button_task4.on_click(on_sign_message_button_clicked)

# --- Display UI ---
display(header_task4)
display(description_task4)

key_gen_box = widgets.VBox([
    sig_alg_dropdown_task4,
    generate_sig_keys_button_task4,
    sig_public_key_output_task4,
    sig_private_key_output_task4
])

signing_box = widgets.VBox([
    message_to_sign_input_task4,
    sign_message_button_task4,
    signature_output_task4
])

main_ui_container_task4 = widgets.VBox([
    key_gen_box,
    widgets.HTML("<hr>"), # Visual separator
    signing_box,
    status_output_task4,
    next_steps_info_task4
], layout={'width': '100%'})

display(main_ui_container_task4)